In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk import FreqDist, ngrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
# download stopwords
nltk.download('stopwords')
import unidecode
import re
import string 

from nltk.corpus import stopwords
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
import torch
from tqdm import trange

In [51]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [68]:
df = pd.read_csv('Data.csv', encoding="latin-1")
df.drop(columns=['Unnamed: 0'], inplace=True)
df.rename(columns={'0':'label', 'a':'text'}, inplace=True)
df

,label,text
0,Thyroid_Cancer,Thyroid surgery in children in a single insti...
1,Thyroid_Cancer,""" The adopted strategy was the same as that us..."
2,Thyroid_Cancer,coronary arterybypass grafting thrombosis ï¬b...
3,Thyroid_Cancer,Solitary plasmacytoma SP of the skull is an u...
4,Thyroid_Cancer,This study aimed to investigate serum matrix ...
...,...,...
496,Colon_Cancer,""" preproof 0c highlights f0b7 propolis..."
497,Colon_Cancer,tetrastigma hemsleyanum diels et gilg t ...
498,Colon_Cancer,neprilysin nep is a neutral endopeptidase it ...
499,Colon_Cancer,covid19 has had an impact on the provision of...


In [69]:
df['text_lw'] = df['text'].str.lower()
df[['text','text_lw']].head(10)

,text,text_lw
0,Thyroid surgery in children in a single insti...,thyroid surgery in children in a single insti...
1,""" The adopted strategy was the same as that us...",""" the adopted strategy was the same as that us..."
2,coronary arterybypass grafting thrombosis ï¬b...,coronary arterybypass grafting thrombosis ï¬b...
3,Solitary plasmacytoma SP of the skull is an u...,solitary plasmacytoma sp of the skull is an u...
4,This study aimed to investigate serum matrix ...,this study aimed to investigate serum matrix ...
5,This study was performed to explore the effec...,this study was performed to explore the effec...
6,This study was performed assess the clinical ...,this study was performed assess the clinical ...
7,Journal of International Medical Research  Th...,journal of international medical research  th...
8,Gastric cancer GC persists as a worldwide pub...,gastric cancer gc persists as a worldwide pub...
9,Scars Burns HealingVolume  reuse guideli...,scars burns healingvolume  reuse guideli...


In [70]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [71]:
sw = stopwords.words('english')

def clean_text(text):
    # remove 
    text = re.sub('\[.*?\]', '', text)
    # remove links
    text = re.sub('https?://\S+|www\.\S+', '', text)
    # remove tags
    text = re.sub('<.*?>+', '', text)
    # remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # remove breaklines
    text = re.sub('\n', '', text)
    # remove numbers
    text = re.sub('\w*\d\w*', '', text)
    
    # remove accent
    text = unidecode.unidecode(text)
    
    # transform text into token
    text_token = nltk.word_tokenize(text)
    
    # remove stopwords
    words = [w for w in text_token if w not in sw]
    
    return ' '.join(words)

df['text_cl'] = df['text_lw'].apply(clean_text)
df[['text','text_lw','text_cl']].head(10)

,text,text_lw,text_cl
0,Thyroid surgery in children in a single insti...,thyroid surgery in children in a single insti...,thyroid surgery children single institution os...
1,""" The adopted strategy was the same as that us...",""" the adopted strategy was the same as that us...",adopted strategy used prior years based four e...
2,coronary arterybypass grafting thrombosis ï¬b...,coronary arterybypass grafting thrombosis ï¬b...,coronary arterybypass grafting thrombosis ! br...
3,Solitary plasmacytoma SP of the skull is an u...,solitary plasmacytoma sp of the skull is an u...,solitary plasmacytoma sp skull uncommon clinic...
4,This study aimed to investigate serum matrix ...,this study aimed to investigate serum matrix ...,study aimed investigate serum matrix metallopr...
5,This study was performed to explore the effec...,this study was performed to explore the effec...,study performed explore effective management b...
6,This study was performed assess the clinical ...,this study was performed assess the clinical ...,study performed assess clinical outcomes elder...
7,Journal of International Medical Research  Th...,journal of international medical research  th...,journal international medical research authors...
8,Gastric cancer GC persists as a worldwide pub...,gastric cancer gc persists as a worldwide pub...,gastric cancer gc persists worldwide public he...
9,Scars Burns HealingVolume  reuse guideli...,scars burns healingvolume  reuse guideli...,scars burns healingvolume reuse guidelinessage...


In [72]:
lemmatizer = WordNetLemmatizer() 

def lemmatize_sentence(text):
    # transform text into token
    text_token = nltk.word_tokenize(text)
    lemmatized_sentence = []
    for word in text_token:
        lemmatized_sentence.append(word)
    return " ".join(lemmatized_sentence)    

In [73]:
df['text_lm'] = df['text_cl'].apply(lemmatize_sentence)
df[['text','text_lw','text_cl', 'text_lm']].head(20)

,text,text_lw,text_cl,text_lm
0,Thyroid surgery in children in a single insti...,thyroid surgery in children in a single insti...,thyroid surgery children single institution os...,thyroid surgery children single institution os...
1,""" The adopted strategy was the same as that us...",""" the adopted strategy was the same as that us...",adopted strategy used prior years based four e...,adopted strategy used prior years based four e...
2,coronary arterybypass grafting thrombosis ï¬b...,coronary arterybypass grafting thrombosis ï¬b...,coronary arterybypass grafting thrombosis ! br...,coronary arterybypass grafting thrombosis ! br...
3,Solitary plasmacytoma SP of the skull is an u...,solitary plasmacytoma sp of the skull is an u...,solitary plasmacytoma sp skull uncommon clinic...,solitary plasmacytoma sp skull uncommon clinic...
4,This study aimed to investigate serum matrix ...,this study aimed to investigate serum matrix ...,study aimed investigate serum matrix metallopr...,study aimed investigate serum matrix metallopr...
5,This study was performed to explore the effec...,this study was performed to explore the effec...,study performed explore effective management b...,study performed explore effective management b...
6,This study was performed assess the clinical ...,this study was performed assess the clinical ...,study performed assess clinical outcomes elder...,study performed assess clinical outcomes elder...
7,Journal of International Medical Research  Th...,journal of international medical research  th...,journal international medical research authors...,journal international medical research authors...
8,Gastric cancer GC persists as a worldwide pub...,gastric cancer gc persists as a worldwide pub...,gastric cancer gc persists worldwide public he...,gastric cancer gc persists worldwide public he...
9,Scars Burns HealingVolume  reuse guideli...,scars burns healingvolume  reuse guideli...,scars burns healingvolume reuse guidelinessage...,scars burns healingvolume reuse guidelinessage...


In [74]:
vectorizer = CountVectorizer(min_df=0.2, max_df=0.8)
docs       = vectorizer.fit_transform(df['text_lm'].tolist())
features   = vectorizer.get_feature_names()

stop_words_ = vectorizer.stop_words_
print(f'Removing {len(stop_words_)} words')

def remove_stopwords(text):
    # transform text into token
    text_token = nltk.word_tokenize(text)
    clean_text = []
    for word in not stop_words_:
        text_token.append(word)
    return " ".join(text_token)    

df['text_sw'] = df['text_lm'].apply(lemmatize_sentence)
df[['text','text_lw','text_cl', 'text_lm', 'text_sw']].head(20)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Removing 141981 words


,text,text_lw,text_cl,text_lm,text_sw
0,Thyroid surgery in children in a single insti...,thyroid surgery in children in a single insti...,thyroid surgery children single institution os...,thyroid surgery children single institution os...,thyroid surgery children single institution os...
1,""" The adopted strategy was the same as that us...",""" the adopted strategy was the same as that us...",adopted strategy used prior years based four e...,adopted strategy used prior years based four e...,adopted strategy used prior years based four e...
2,coronary arterybypass grafting thrombosis ï¬b...,coronary arterybypass grafting thrombosis ï¬b...,coronary arterybypass grafting thrombosis ! br...,coronary arterybypass grafting thrombosis ! br...,coronary arterybypass grafting thrombosis ! br...
3,Solitary plasmacytoma SP of the skull is an u...,solitary plasmacytoma sp of the skull is an u...,solitary plasmacytoma sp skull uncommon clinic...,solitary plasmacytoma sp skull uncommon clinic...,solitary plasmacytoma sp skull uncommon clinic...
4,This study aimed to investigate serum matrix ...,this study aimed to investigate serum matrix ...,study aimed investigate serum matrix metallopr...,study aimed investigate serum matrix metallopr...,study aimed investigate serum matrix metallopr...
5,This study was performed to explore the effec...,this study was performed to explore the effec...,study performed explore effective management b...,study performed explore effective management b...,study performed explore effective management b...
6,This study was performed assess the clinical ...,this study was performed assess the clinical ...,study performed assess clinical outcomes elder...,study performed assess clinical outcomes elder...,study performed assess clinical outcomes elder...
7,Journal of International Medical Research  Th...,journal of international medical research  th...,journal international medical research authors...,journal international medical research authors...,journal international medical research authors...
8,Gastric cancer GC persists as a worldwide pub...,gastric cancer gc persists as a worldwide pub...,gastric cancer gc persists worldwide public he...,gastric cancer gc persists worldwide public he...,gastric cancer gc persists worldwide public he...
9,Scars Burns HealingVolume  reuse guideli...,scars burns healingvolume  reuse guideli...,scars burns healingvolume reuse guidelinessage...,scars burns healingvolume reuse guidelinessage...,scars burns healingvolume reuse guidelinessage...


In [75]:
df['label']= df['label'].replace(['Thyroid_Cancer', 'Colon_Cancer', 'Lung_Cancer'],[0,1,2])

In [76]:
text = df.text.values
labels = df.label.values

In [77]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-v1.1', do_lower_case=True)

In [78]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
    return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 16,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids']) 
    attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\abhis\AppData\Roaming\Python\Python39\site-packages\transformers\tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [79]:
val_ratio = 0.2
batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [80]:
def b_tp(preds, labels):
    return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
    return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
    return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
    return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
    preds = np.argmax(preds, axis = 1).flatten()
    labels = labels.flatten()
    tp = b_tp(preds, labels)
    tn = b_tn(preds, labels)
    fp = b_fp(preds, labels)
    fn = b_fn(preds, labels)
    b_accuracy = (tp + tn) / len(labels)
    b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
    b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
    b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
    return b_accuracy, b_precision, b_recall, b_specificity

In [81]:
# Load the BertForSequenceClassification model
model = BertForSequenceClassification.from_pretrained(
    'dmis-lab/biobert-v1.1',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5.
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [82]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 4

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')

Epoch:  25%|███████████████████▎                                                         | 1/4 [00:27<01:23, 27.96s/it]


	 - Train loss: 0.6812
	 - Validation Accuracy: 0.6679
	 - Validation Precision: 0.8357
	 - Validation Recall: 0.3282
	 - Validation Specificity: 0.9161



Epoch:  50%|██████████████████████████████████████▌                                      | 2/4 [00:56<00:56, 28.36s/it]


	 - Train loss: 0.4942
	 - Validation Accuracy: 0.7857
	 - Validation Precision: 0.7507
	 - Validation Recall: 0.7602
	 - Validation Specificity: 0.8041



Epoch:  75%|█████████████████████████████████████████████████████████▊                   | 3/4 [01:24<00:28, 28.13s/it]


	 - Train loss: 0.2799
	 - Validation Accuracy: 0.7768
	 - Validation Precision: 0.7100
	 - Validation Recall: 0.8112
	 - Validation Specificity: 0.7531



Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 4/4 [01:52<00:00, 28.02s/it]


	 - Train loss: 0.1526
	 - Validation Accuracy: 0.7768
	 - Validation Precision: 0.7563
	 - Validation Recall: 0.7080
	 - Validation Specificity: 0.8259

